# Import a mosain into Geoserver
- info source: https://pypi.org/project/geoserver-rest/
- Things that we need:
https://github.com/geosolutions-it/geotools

In [1]:
import os
from zipfile import ZipFile
from geo.Geoserver import Geoserver
import pandas as pd

In [2]:
"""WARNING, BE CAREFUL WITH WHAT DO YOU IMPORT, MAKE SURE ARE TIFS"""
path = r"Z:\data\ruben\global_forest_watch_data_adapted"

In [3]:
"""Take the folders in case there is more than one folder"""
def get_folder_list(path):
    Folder_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for folder in os.listdir(path): # [1:]
        if os.path.isdir(folder):
            if folder not in Folder_list:
                Folder_list.append(os.path.join(path, folder).replace("\\","/"))
        else:
            pass
    if not Folder_list:
        Folder_list = [path]
        return Folder_list
    else:
        return Folder_list


In [1]:
# We need to crete with geotools the required parameters to import into Geoserver
# Here are some tips to publish: https://github.com/geosolutions-it/geoserver-restconfig
# Here are the instructions: https://github.com/geosolutions-it/geotools 
# Here are the properties of a mosaic: https://docs.geotools.org/latest/userguide/library/coverage/mosaic.html

In [8]:
def create_zip(folder):
    """Creates a zip inside the folder for every raster that is inside"""
    # Get the parent directory of the input folder
    parent_folder = os.path.dirname(folder)
    #We create the object with the name outside the scope of the folder
    zip_file_path = os.path.join(parent_folder, os.path.basename(folder) + ".zip")
    zipObj = ZipFile(zip_file_path, 'w')
    # Iterate over the files in the folder
    for file_name in os.listdir(folder):
        file_path = os.path.join(folder, file_name)
        if os.path.isfile(file_path) and (file_name.endswith(".tif") or file_name.endswith(".tiff")):
            zipObj.write(file_path, file_name)
    return zip_file_path

In [ ]:
folder_list = get_folder_list(path)

In [9]:
for folder in folder_list:
    zip_path_list = []
    zip_file_path = create_zip(folder)
    # Store the path of the zips
    zip_path_list.append(zip_file_path)

In [ ]:
#Initialize the library
"""Production-geoserver"""
#geo = Geoserver('https://integratedmodelling.org/dev-geoserver', username='admin', password='curd-h0cod')
"""Development-geoserver"""
geo = Geoserver('http://192.168.250.122:8600/geoserver', username='', password='')

# Name of the workspace
workspace_name = "ruben"

In [ ]:
# Create workspace if it doesn't exist
workspace = geo.get_workspace(workspace_name)
if workspace is None:
    geo.create_workspace(workspace_name, workspace_name)

In [ ]:
def geoserver_raster_batch(geo, file_path, workspace_name):
    name = os.path.basename(file_path).replace('.tif','') # Store the name without .tif
    # Upload raster data to the geoserver
    geo.create_imagemosaic(lyr_name=name, path=file_path, workspace = workspace_name)
    return print("Imported: " + file_path)

for file_path in zip_path_list:
    geoserver_raster_batch(geo, file_path, workspace_name)

In [ ]:
for file_path in zip_path_list:
    geoserver_raster_batch(geo, file_path, workspace_name)

In [ ]:
for file in os.listdir(path):
    geoserver_raster_batch(geo, path, file, workspace_name)


#geo.create_coveragestore(lyr_name="global-accessibility-2021", path="C:/Users/ruben.crespo/Documents/03_tests/global_accesibility/access_50k/store/global-accessibility-2021.tif", workspace='im-data-global-demography')